<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Business Analytics e Machine Learning Para Projetos de Data Science</font>
## <font color='blue'>Projeto 13</font>
#### <font color='blue'>Sistema de Recomendação de Manutenção Preventiva Integrado com IoT Para Redução de Paradas Não Planejadas</font>

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

Nota: Pode ser necessário remover o pacote chamado imblearn com o comando: pip uninstall imblearn

https://pypi.org/project/imblearn/

O pacote a ser usado é imbalanced-learn, mas carregamos como imblearn!

O procedimento de instalação de pacotes e dependências está no arquivo LEIAME.txt

In [2]:
#!pip install -q imbalanced-learn

In [3]:
#!pip install -q lightgbm

Nota 2: caso tenha problema ao importar o lightgbm (do tipo `OSError: libgomp.so.1: cannot open shared object file: No such file or directory`) a documentação do lightgbm diz para instalar a biblioteca OpenMP separadamente. No caso do macbook faça:

```brew install libomp```

In [2]:
# Imports
import joblib
import sklearn
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [5]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando os Dados e Verificando Proporção de Classe

In [3]:
# Carrega o dataset
df_dsa = pd.read_csv('dataset.csv')

In [4]:
# Shape
df_dsa.shape

(10000, 6)

In [5]:
# Amostra
df_dsa.head()

,vibracao,temperatura,pressao,umidade,horas_trabalho,manutencao_necessaria
0,0.250951,92.419225,100.311847,67.596275,7499,0
1,0.895355,69.132552,96.137413,70.454398,600,0
2,0.564789,66.456903,93.642299,31.822434,6919,0
3,0.853165,81.967579,101.924996,46.543886,4032,1
4,2.143944,60.097525,97.527537,50.129838,8036,0


In [6]:
# Info
df_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vibracao               10000 non-null  float64
 1   temperatura            10000 non-null  float64
 2   pressao                10000 non-null  float64
 3   umidade                10000 non-null  float64
 4   horas_trabalho         10000 non-null  int64  
 5   manutencao_necessaria  10000 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 468.9 KB


In [7]:
# Proporção de classe
df_dsa.manutencao_necessaria.value_counts()

manutencao_necessaria
1    5517
0    4483
Name: count, dtype: int64

- 1 é a classe positiva (foi necessário manutenção na máquina)
- 0 é a classe negativa (não foi necessário manutenção na máquina)

## Preparação dos Dados

In [9]:
# Separar variáveis explicativas (X) e variável alvo (y)
X = df_dsa.drop('manutencao_necessaria', axis = 1)
y = df_dsa['manutencao_necessaria']

In [10]:
# Dividir os dados em conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Por que aplicamos balanceamento de classe somente em dados de treino?

Leia o ebook **O Que é e Como Tratar o Desbalanceamento de Classe?** no Capítulo 21 do curso.

## Opção 1 - Ajuste de Pesos no Modelo (Sem Reamostragem)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [11]:
# Padronizar os dados 
scaler_v1 = StandardScaler()
X_train_scaled = scaler_v1.fit_transform(X_train)
X_test_scaled = scaler_v1.transform(X_test)

In [12]:
# Instanciar o modelo com ajuste de pesos
modelo_v1 = RandomForestClassifier(n_estimators = 100, 
                                   random_state = 42, 
                                   class_weight = 'balanced')

In [13]:
%%time
modelo_v1.fit(X_train_scaled, y_train)

CPU times: user 909 ms, sys: 64.8 ms, total: 974 ms
Wall time: 1.12 s


RandomForestClassifier(class_weight='balanced', random_state=42)

In [14]:
# Fazer previsões no conjunto de teste
y_pred = modelo_v1.predict(X_test_scaled)
y_pred_proba = modelo_v1.predict_proba(X_test_scaled)[:, 1]

In [15]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.60%
AUC-ROC: 98.82%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98       876
           1       0.99      0.98      0.99      1124

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [16]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v1.pkl'
scaler_file = 'padronizadores/scaler_v1.pkl'

joblib.dump(modelo_v1, model_file)
joblib.dump(scaler_v1, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v1.pkl
Scaler salvo em padronizadores/scaler_v1.pkl


## Opção 2 - Subamostragem da Classe Majoritária (Undersampling)

In [17]:
# Proporção de classe
df_dsa.manutencao_necessaria.value_counts()

manutencao_necessaria
1    5517
0    4483
Name: count, dtype: int64

In [18]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [19]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao_necessaria == 1]
df_minoritaria = train_data[train_data.manutencao_necessaria == 0]

Lembre-se que aplicamos reamostragem nos dados de treino!

In [20]:
len(df_majoritaria)

4393

In [21]:
len(df_minoritaria)

3607

In [22]:
# Aplicar subamostragem da classe majoritária no conjunto de treino
df_majoritaria_subamostrada = resample(df_majoritaria,
                                       replace = False,    
                                       n_samples = len(df_minoritaria),  # Igualar o número da classe minoritária
                                       random_state = 42)

In [23]:
# Combinar as classes minoritária e majoritária subamostrada
train_data_balanceado = pd.concat([df_majoritaria_subamostrada, df_minoritaria])

In [24]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao_necessaria', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao_necessaria']

In [25]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao_necessaria
1    3607
0    3607
Name: count, dtype: int64


In [26]:
# Padronizar os dados
scaler_v2 = StandardScaler()
X_train_scaled = scaler_v2.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v2.transform(X_test)

In [27]:
# Instanciar e treinar o modelo
modelo_v2 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [28]:
%%time
modelo_v2.fit(X_train_scaled, y_train_balanceado)

CPU times: user 841 ms, sys: 75.8 ms, total: 917 ms
Wall time: 990 ms


RandomForestClassifier(random_state=42)

In [29]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v2.predict(X_test_scaled)
y_pred_proba = modelo_v2.predict_proba(X_test_scaled)[:, 1]

In [30]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.20%
AUC-ROC: 98.28%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       876
           1       0.99      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [33]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v2.pkl'
scaler_file = 'padronizadores/scaler_v2.pkl'

joblib.dump(modelo_v2, model_file)
joblib.dump(scaler_v2, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v2.pkl
Scaler salvo em padronizadores/scaler_v2.pkl


## Opção 3 - Superamostragem da Classe Minoritária (Oversampling)

In [31]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [32]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao_necessaria == 1]
df_minoritaria = train_data[train_data.manutencao_necessaria == 0]

Lembre-se que aplicamos reamostragem nos dados de treino!

In [33]:
len(df_majoritaria)

4393

In [34]:
len(df_minoritaria)

3607

In [35]:
# Superamostragem da classe minoritária no conjunto de treino
df_minoritaria_superamostrada = resample(df_minoritaria,
                                         replace = True,     
                                         n_samples = len(df_majoritaria), # Igualar ao número da classe majoritária
                                         random_state = 42)

In [36]:
# Combinar as classes majoritária e minoritária superamostrada
train_data_balanceado = pd.concat([df_majoritaria, df_minoritaria_superamostrada])

In [37]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao_necessaria', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao_necessaria']

In [38]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao_necessaria
1    4393
0    4393
Name: count, dtype: int64


In [39]:
# Padronizar os dados
scaler_v3 = StandardScaler()
X_train_scaled = scaler_v3.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v3.transform(X_test)

In [40]:
# Criar o modelo
modelo_v3 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [41]:
%%time
modelo_v3.fit(X_train_scaled, y_train_balanceado)

CPU times: user 881 ms, sys: 47.8 ms, total: 929 ms
Wall time: 955 ms


RandomForestClassifier(random_state=42)

In [42]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v3.predict(X_test_scaled)
y_pred_proba = modelo_v3.predict_proba(X_test_scaled)[:, 1]

In [43]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.25%
AUC-ROC: 98.95%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       876
           1       0.98      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [44]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v3.pkl'
scaler_file = 'padronizadores/scaler_v3.pkl'

joblib.dump(modelo_v3, model_file)
joblib.dump(scaler_v3, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v3.pkl
Scaler salvo em padronizadores/scaler_v3.pkl


## Opção 4 - Balanceamento Automático com SMOTE

SMOTE (Synthetic Minority Over-sampling Technique): SMOTE é uma técnica de geração de novas amostras para a classe minoritária com base nas distâncias entre os pontos. Essa operação é sensível às distâncias entre os dados e se os dados não forem escalonados, variáveis com magnitudes maiores (como "horas_trabalho" comparado a "vibracao") podem influenciar mais na geração das novas amostras.

In [45]:
# Padronização
scaler_v4 = StandardScaler()
X_train_scaled = scaler_v4.fit_transform(X_train)
X_test_scaled = scaler_v4.transform(X_test)

In [46]:
# Cria o SMOTE
smote = SMOTE(random_state = 42)

In [47]:
# Treinar e aplicar SMOTE no conjunto de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [48]:
len(X_train_smote)

8786

In [49]:
# Criar o modelo
modelo_v4 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [50]:
%%time
modelo_v4.fit(X_train_smote, y_train_smote)

CPU times: user 1.03 s, sys: 64 ms, total: 1.1 s
Wall time: 1.14 s


RandomForestClassifier(random_state=42)

In [51]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v4.predict(X_test_scaled)
y_pred_proba = modelo_v4.predict_proba(X_test_scaled)[:, 1]

In [52]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.80%
AUC-ROC: 99.02%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       876
           1       0.99      0.98      0.99      1124

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [53]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v4.pkl'
scaler_file = 'padronizadores/scaler_v4.pkl'

joblib.dump(modelo_v4, model_file)
joblib.dump(scaler_v4, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v4.pkl
Scaler salvo em padronizadores/scaler_v4.pkl


## Opção 5 - Balanceamento Automático com SMOTE e Mudança de Algoritmo

ATENÇÃO!!! OBSERVE A ORDEM DAS TAREFAS DE PADRONIZAÇÃO E SMOTE!

A ordem correta entre padronização e SMOTE depende da natureza das operações que cada uma realiza:

SMOTE (Synthetic Minority Over-sampling Technique): SMOTE é uma técnica de geração de novas amostras para a classe minoritária com base nas distâncias entre os pontos. Essa operação é sensível às distâncias entre os dados e se os dados não forem escalonados, variáveis com magnitudes maiores (como "horas_trabalho" comparado a "vibracao") podem influenciar mais na geração das novas amostras.

Padronização (Standardization): A padronização escala as variáveis para terem média 0 e desvio padrão 1, o que remove a diferença de magnitude entre as variáveis. Como a SMOTE depende de distâncias, é importante que os dados estejam padronizados antes da aplicação do SMOTE, para garantir que todas as variáveis tenham igual influência na geração de novas amostras.

Você deve aplicar a padronização primeiro e o SMOTE depois. Isso garante que as distâncias entre os pontos, que o SMOTE usa para gerar novos exemplos, não sejam distorcidas por variáveis com escalas diferentes.

> SMOTE Primeiro e Padronização Depois!

In [54]:
%%time

# Criar o SMOTE
smote = SMOTE(random_state = 42)

# Treinar e aplicar SMOTE no conjunto de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Padronização
scaler_v5 = StandardScaler()
X_train_scaled = scaler_v5.fit_transform(X_train_smote)
X_test_scaled = scaler_v5.transform(X_test)

# Criar o modelo
modelo_v5 = lgb.LGBMClassifier(random_state = 42)

# Treinar o modelo com o conjunto de treino balanceado via SMOTE
modelo_v5.fit(X_train_scaled, y_train_smote)

# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v4.predict(X_test_scaled)
y_pred_proba = modelo_v4.predict_proba(X_test_scaled)[:, 1]

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 4393, number of negative: 4393
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000360 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 8786, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Acurácia: 96.30%
AUC-ROC: 98.52%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       876
           1       0.99      0.95      0.97      1124

    accuracy                           0.96      2000
   macro avg       0.96      0.97      0.96      2000
weighted avg       0.96      0.96      0.96      2000

CPU times: user 225 ms, sys: 430 ms, total: 655 ms
Wall time: 645 ms


> Padronização Primeiro e SMOTE Depois!

In [55]:
%%time

# Padronizar as variáveis
scaler_v5 = StandardScaler()
X_train_scaled = scaler_v5.fit_transform(X_train)
X_test_scaled = scaler_v5.transform(X_test)

# Criar o SMOTE
smote = SMOTE(random_state = 42)

# Aplicar SMOTE no conjunto de treino para lidar com o desbalanceamento
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Criar o modelo LightGBM
modelo_v5 = lgb.LGBMClassifier(random_state = 42)

# Treinar o modelo LightGBM
modelo_v5.fit(X_train_smote, y_train_smote)

# Avaliar o modelo no conjunto de teste original
y_pred = modelo_v5.predict(X_test_scaled)
y_pred_proba = modelo_v5.predict_proba(X_test_scaled)[:, 1]

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 4393, number of negative: 4393
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000215 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 8786, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Acurácia: 92.40%
AUC-ROC: 97.47%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.86      0.98      0.92       876
           1       0.98      0.88      0.93      1124

    accuracy                           0.92      2000
   macro avg       0.92      0.93      0.92      2000
weighted avg       0.93      0.92      0.92      2000

CPU times: user 229 ms, sys: 365 ms, total: 594 ms
Wall time: 368 ms


In [56]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v5.pkl'
scaler_file = 'padronizadores/scaler_v5.pkl'

joblib.dump(modelo_v5, model_file)
joblib.dump(scaler_v5, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v5.pkl
Scaler salvo em padronizadores/scaler_v5.pkl


## Seleção de Modelo

Versão 1 do Modelo:

- Acurácia: 98.70%
- AUC-ROC: 98.78%

Versão 2 do Modelo:

- Acurácia: 98.20%
- AUC-ROC: 98.28%

Versão 3 do Modelo:

- Acurácia: 98.25%
- AUC-ROC: 98.96%

Versão 4 do Modelo:

- Acurácia: 98.85%
- AUC-ROC: 99.03%

Versão 5 do Modelo:

SMOTE Primeiro e Padronização Depois!
- Acurácia: 96.30%
- AUC-ROC: 98.53%

Padronização Primeiro e SMOTE Depois! (Essa é a ordem ideal)
- Acurácia: 92.40%
- AUC-ROC: 97.47%

**Qual modelo você escolheria? Justifique sua resposta!**

Um modelo de Machine Learning é resultado desta equação: **algoritmo + dados**! Ou seja, um modelo simples é aquele que oferece boa performance com o mínimo de mudança nos dados e o mínimo de mudança no algoritmo. Com base nesse critério, o modelo versão 1 é a melhor opção e será nossa escolha para o deploy!

A versão 1 do modelo foi a que apresentou o maior equilíbrio entre:

**Capacidade de Generalização / Performance / Simplicidade / Interpretabilidade**

Nosso desafio é sempre encontrar o modelo que consiga equilibrar esses 4 elementos! E isso é quase uma arte!

## Testando o Deploy do Modelo

In [60]:
# Função para recomendar manutenção baseada em novos dados de sensores IoT
def recomendacao_manutencao(novo_dado):
    
    # Definir os nomes das colunas conforme o scaler foi ajustado
    colunas = ['vibracao', 'temperatura', 'pressao', 'umidade', 'horas_trabalho']
    
    # Converter os novos dados para DataFrame com os nomes de colunas corretos
    novo_dado_df = pd.DataFrame([novo_dado], columns = colunas)
    
    # Aplicar o scaler aos novos dados
    novo_dado_scaled = scaler_v1.transform(novo_dado_df)
    
    # Fazer a previsão
    predicao = modelo_v1.predict(novo_dado_scaled)
    
    if predicao == 1:
        return "Recomendação: Realizar manutenção."
    else:
        return "Recomendação: Nenhuma manutenção necessária."

In [61]:
# Exemplo de novos dados de sensores IoT 
novos_dados_1 = [0.5, 80, 102, 45, 8000]
print(recomendacao_manutencao(novos_dados_1))

Recomendação: Realizar manutenção.


In [62]:
# Exemplo de novos dados de sensores IoT 
novos_dados_2 = [0.89, 92, 96, 70, 600]
print(recomendacao_manutencao(novos_dados_2))

Recomendação: Nenhuma manutenção necessária.


**Vamos agora realizar o deploy via app web com Streamlit.**

In [63]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [64]:
#!pip show imbalanced-learn | grep Version

In [65]:
#%watermark -v -m

In [66]:
#%watermark --iversions

# Fim